# viresclient API

## Introduction

The `viresclient` Python package allows to connect to the VirES server to download [Swarm](https://earth.esa.int/web/guest/missions/esa-operational-eo-missions/swarm) data and data calculated using magnetic models. This notebook describes the main classes and functions included in the the package.

For more information, see: https://viresclient.readthedocs.io/

<a id="top"/>

## Contents
- [Access token configuration](#access_token_configuration)
- [Send requests to the server - SwarmRequest](#SwarmRequest)
    - [Get the available collections](#SwarmRequest.available_collections)
    - [Get the available measurements](#SwarmRequest.available_measurements)
    - [Get the available auxiliaries](#SwarmRequest.available_auxiliaries)
    - [Get the available magnetic models](#SwarmRequest.available_models)
    - [Get information about one or mode models](#SwarmRequest.get_model_info)
    - [Get the orbit number](#SwarmRequest.get_orbit_number)
    - [Get times for orbits](#SwarmRequest.get_times_for_orbits)
    - [Set collections](#SwarmRequest.set_collection)
    - [Set products](#SwarmRequest.set_products)
    - [Set/clear filters](#SwarmRequest.set_range_filter)
    - [Send request to the server](#SwarmRequest.get_between)
- [Handle downloaded data - ReturnedData](#ReturnedData)
    - [Get the list of source data](#ReturnedData.sources)
    - [ReturnedData contents](#ReturnedData.contents)
    - [Get type of downloaded data files](#ReturnedData.filetype)
    - [Get list of magnetic models used during calculations](#ReturnedData.magnetic_models)
    - [Get list of filters applied to the request](#ReturnedData.range_filters)
    - [Convert ReturnedData to Pandas DataFrame](#ReturnedData.as_dataframe)
    - [Convert ReturnedData to xarray Dataset](#ReturnedData.as_xarray)
    - [Save downloaded data to a file](#ReturnedData.to_file)
    - [Save downloaded data to multiple files](#ReturnedData.to_files)
- [Handle downloaded temporary file - ReturnedDataFile](#ReturnedDataFile)
    - [Get NamedTemporaryFile associated to a ReturnedDataFile](#ReturnedDataFile._file)
    - [Get type of the downloaded data file](#ReturnedDataFile.filetype)
    - [Convert ReturnedDataFile to Pandas DataFrame](#ReturnedDataFile.as_dataframe)
    - [Convert ReturnedDataFile to xarray Dataset](#ReturnedDataFile.as_xarray)
    - [Save ReturnedDataFile object to a file](#ReturnedDataFile.to_file)
- [Handle viresclient configuration - ClientConfig](#ClientConfig)
    - [Get path of the configuration file](#ClientConfig.path)
    - [Get or set the default URL](#ClientConfig.default_url)
    - [Set site configuration](#ClientConfig.set_site_config)
    - [Get site configuration](#ClientConfig.get_site_config)
    - [Save configuration](#ClientConfig.save)
- [Upload data to the server - DataUpload](#DataUpload)
    - [Upload a file to the server](#DataUpload.post)
    - [Get the identifier(s) of the uploaded file(s)](#DataUpload.ids)
    - [Get info about the uploaded file](#DataUpload.get)
    - [Set constant parameters to the uploaded file](#DataUpload.set_constant_parameters)
    - [Get constant parameters](#DataUpload.get_constant_parameters)
    - [Delete a specific uploaded file](#DataUpload.delete)
    - [Delete the uploaded files](#DataUpload.clear)

[TOP](#top)

<a id="access_token_configuration"/>

## Access token configuration

Before using the client, you need to set the access token. This can be done using the `set_token()` function:

```python
set_token(url='https://vires.services/ows', token=None, set_default=False)
```

**Arguments**:

- **url** (*str*, optional): server URL (default value: `https://vires.services/ows`).
- **token** (*str*, optional): token string obtained from the VirES access token management page (https://vires.services/accounts/tokens/). If this parameter is not set, the user is prompted to enter its value interactively.
- **set_default** (*bool*, optional): if `True`, the server identified by *url* is configured as default server (default `False`).

Since the *url* parameter is by default set to the correct value you don't need to specify it. You can avoid also to specify the *token* parameter because this value will be asked interactively. Moreover, setting `set_default=True` you can configure this URL as default in the configuration file and avoid to specify it while sending requests to the server.

First, the function must be imported from the `viresclient` package:

In [1]:
from viresclient import set_token

Now it can be executed:

In [ ]:
set_token(set_default=True)

Setting access token for https://vires.services/ows ...
Generate a token at https://vires.services/accounts/tokens/


[TOP](#top)

<a id="SwarmRequest"/>

## Send requests to the server - SwarmRequest

The `SwarmRequest` object allows to create a request to the server and to download data according to the input parameters.

```python
SwarmRequest(url=None, username=None, password=None, token=None, config=None, logging_level='NO_LOGGING')
```

**Attributes**:

- **url** (*str*, optional): if not provided, the request will be sent to the default URL set in the `~/.viresclient.ini` configuration file.
- **username** (*str*, optional): username. The usage of username and password is deprecated and will be removed in future releases.
- **password** (*str*, optional): password. The usage of username and password is deprecated and will be removed in future releases.
- **token** (*str*, optional): token string obtained from the VirES access token management page (https://vires.services/accounts/tokens/). If this parameter is not specified and it is not set in the configuration file, the user is prompted to enter its value interactively.
- **config** (*str* or *ClientConfig*, optional): viresclient configuration. By default, it is read from `~/.viresclient.ini`.
- **logging_level** (*str*, optional): set the logging level. Allowed values are: `DEBUG`, `INFO`, `WARNING`, `ERROR`, `NO_LOGGING` (default).

Let's import the class from the `viresclient` package:

In [2]:
from viresclient import SwarmRequest

After configuring the access token using the `set_token()` function (see [Access token configuration](#access_token_configuration)) , the `SwarmRequest` object can be created as follows:

In [ ]:
request = SwarmRequest()

The access token for the `https://vires.services/ows` default URL configured in the `~/.viresclient.ini` file is automatically retrieved:

```ini
[https://vires.services/ows]
token = <token>

[default]
url = https://vires.services/ows
```

If the default server URL is not configured (i.e. the "default" section is not present), you must specify the server URL:

In [ ]:
request = SwarmRequest('https://vires.services/ows')

`SwarmRequest` object has the following methods:

- `SwarmRequest.available_collections()`
- `SwarmRequest.available_measurements()`
- `SwarmRequest.available_auxiliaries()`
- `SwarmRequest.available_models()`
- `SwarmRequest.get_model_info()`
- `SwarmRequest.get_orbit_number()`
- `SwarmRequest.get_times_for_orbits()`
- `SwarmRequest.set_collection()`
- `SwarmRequest.set_products()`
- `SwarmRequest.set_range_filter()`
- `SwarmRequest.clear_range_filter()`
- `SwarmRequest.get_between()`



[TOP](#top)

<a id="SwarmRequest.available_collections"/>

### Get the available collections

Swarm data are organized in *collections*. Each collection is related to a Swarm file type (e.g. collection *SW_OPER_MAGA_LR_1B* is related to file type *MAGA_LR_1B*). The list of the available collections are provided invoking the `SwarmRequest.available_collections()` method.

```python
SwarmRequest.available_collections(details=True)
```

**Arguments**:

- **details** (*bool*, optional): if `True` (default), the method prints the list of all the available collections and the related details. If `False`, it returns the available collections as a *list*.

Example: print the list of the available collections and their details:

In [5]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get the available collections
request.available_collections()

General References:
 Swarm Data Handbook, https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook 
 The Swarm Satellite Constellation Application and Research Facility (SCARF) and Swarm data products, https://doi.org/10.5047/eps.2013.07.001 
 Swarm Science Data Processing and Products (2013), https://link.springer.com/journal/40623/65/11/page/1 
 Special issue “Swarm science results after 2 years in space (2016), https://www.springeropen.com/collections/swsr 
 Earth's Magnetic Field: Understanding Geomagnetic Sources from the Earth's Interior and its Environment (2017), https://link.springer.com/journal/11214/206/1/page/1 

MAG
   SW_OPER_MAGA_LR_1B
   SW_OPER_MAGB_LR_1B
   SW_OPER_MAGC_LR_1B
 https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook/level-1b-product-definitions#MAGX_LR_1B_Product 

EFI
   SW_OPER_EFIA_LP_1B
   SW_OPER_EFIB_LP_1B
   SW_OPER_EFIC_LP_1B
 https://earth.esa.int/web/guest/missions/esa-eo-missions/swarm/data-handbook/l

Example: get the list of available collections without the details:

In [6]:
request.available_collections(details=False)

['SW_OPER_MAGA_LR_1B',
 'SW_OPER_MAGB_LR_1B',
 'SW_OPER_MAGC_LR_1B',
 'SW_OPER_EFIA_LP_1B',
 'SW_OPER_EFIB_LP_1B',
 'SW_OPER_EFIC_LP_1B',
 'SW_OPER_IBIATMS_2F',
 'SW_OPER_IBIBTMS_2F',
 'SW_OPER_IBICTMS_2F',
 'SW_OPER_TECATMS_2F',
 'SW_OPER_TECBTMS_2F',
 'SW_OPER_TECCTMS_2F',
 'SW_OPER_FACATMS_2F',
 'SW_OPER_FACBTMS_2F',
 'SW_OPER_FACCTMS_2F',
 'SW_OPER_FAC_TMS_2F',
 'SW_OPER_EEFATMS_2F',
 'SW_OPER_EEFBTMS_2F',
 'SW_OPER_IPDAIRR_2F',
 'SW_OPER_IPDBIRR_2F',
 'SW_OPER_IPDCIRR_2F']

[TOP](#top)

<a id="SwarmRequest.available_measurements"/>

### Get the available measurements

It is possible to get the available measurements using the `SwarmRequest.available_measurements()` method:

```python
SwarmRequest.available_measurements(collection=None)
```

**Arguments**:

- **collection** (*str*, optional). If indicated, returns the available measurements for this collection as a *list*. It can be set to: `MAG`, `EFI`, `IBI`, `TEC`, `FAC`, `EEF`, `IPD` or one of the collections returned by the *available_collections* method. If not indicated, it returns the available measurements for all the collections as a *dict*.

Example: get the list of measurements for the `MAG` collections:

In [7]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get the available measurements
request.available_measurements('MAG')

['F',
 'dF_AOCS',
 'dF_other',
 'F_error',
 'B_VFM',
 'B_NEC',
 'dB_Sun',
 'dB_AOCS',
 'dB_other',
 'B_error',
 'q_NEC_CRF',
 'Att_error',
 'Flags_F',
 'Flags_B',
 'Flags_q',
 'Flags_Platform',
 'ASM_Freq_Dev']

Setting `collection=SW_OPER_MAGA_LR_1B` we obtain the same result:

In [8]:
request.available_measurements('SW_OPER_MAGA_LR_1B')

['F',
 'dF_AOCS',
 'dF_other',
 'F_error',
 'B_VFM',
 'B_NEC',
 'dB_Sun',
 'dB_AOCS',
 'dB_other',
 'B_error',
 'q_NEC_CRF',
 'Att_error',
 'Flags_F',
 'Flags_B',
 'Flags_q',
 'Flags_Platform',
 'ASM_Freq_Dev']

Example: get all the available measurements:

In [9]:
request.available_measurements()

{'MAG': ['F',
  'dF_AOCS',
  'dF_other',
  'F_error',
  'B_VFM',
  'B_NEC',
  'dB_Sun',
  'dB_AOCS',
  'dB_other',
  'B_error',
  'q_NEC_CRF',
  'Att_error',
  'Flags_F',
  'Flags_B',
  'Flags_q',
  'Flags_Platform',
  'ASM_Freq_Dev'],
 'EFI': ['U_orbit',
  'Ne',
  'Ne_error',
  'Te',
  'Te_error',
  'Vs',
  'Vs_error',
  'Flags_LP',
  'Flags_Ne',
  'Flags_Te',
  'Flags_Vs'],
 'IBI': ['Bubble_Index',
  'Bubble_Probability',
  'Flags_Bubble',
  'Flags_F',
  'Flags_B',
  'Flags_q'],
 'TEC': ['GPS_Position',
  'LEO_Position',
  'PRN',
  'L1',
  'L2',
  'P1',
  'P2',
  'S1',
  'S2',
  'Elevation_Angle',
  'Absolute_VTEC',
  'Absolute_STEC',
  'Relative_STEC',
  'Relative_STEC_RMS',
  'DCB',
  'DCB_Error'],
 'FAC': ['IRC',
  'IRC_Error',
  'FAC',
  'FAC_Error',
  'Flags',
  'Flags_F',
  'Flags_B',
  'Flags_q'],
 'EEF': ['EEF', 'RelErr', 'flags'],
 'IPD': ['Ne',
  'Te',
  'Background_Ne',
  'Foreground_Ne',
  'PCP_flag',
  'Grad_Ne_at_100km',
  'Grad_Ne_at_50km',
  'Grad_Ne_at_20km',
  'Grad

[TOP](#top)

<a id="SwarmRequest.available_auxiliaries"/>

### Get the available auxiliaries

It is possible to get the available auxiliaries using the `SwarmRequest.available_auxiliaries()` method:

```python
viresclient.SwarmRequest.available_auxiliaries()
```

This method does not accept any argument and provides the available auxiliaries as a list.

Example: get the list of the available auxiliaries:

In [10]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get the list of available auxiliaries
request.available_auxiliaries()

['Timestamp',
 'Latitude',
 'Longitude',
 'Radius',
 'Spacecraft',
 'OrbitDirection',
 'QDOrbitDirection',
 'SyncStatus',
 'Kp10',
 'Kp',
 'Dst',
 'F107',
 'IMF_BY_GSM',
 'IMF_BZ_GSM',
 'IMF_V',
 'F10_INDEX',
 'OrbitSource',
 'OrbitNumber',
 'AscendingNodeTime',
 'AscendingNodeLongitude',
 'QDLat',
 'QDLon',
 'QDBasis',
 'MLT',
 'SunDeclination',
 'SunHourAngle',
 'SunRightAscension',
 'SunAzimuthAngle',
 'SunZenithAngle',
 'SunLongitude',
 'SunVector',
 'DipoleAxisVector',
 'NGPLatitude',
 'NGPLongitude',
 'DipoleTiltAngle',
 'UpwardCurrent',
 'TotalCurrent',
 'DivergenceFreeCurrentFunction']

[TOP](#top)

<a id="SwarmRequest.available_models"/>

### Get the available magnetic models

The available magnetic models can be obtained using the `SwarmRequest.available_models()` method:

```python
viresclient.SwarmRequest.available_models(param=None, details=True, nice_output=True)
```

**Arguments**:

- **param** (*str*, optional): it can be set to one of `F`, `C`, `D`, `MCO`, `MLI`, `MMA`, `MIO` to filter all the available magnetic models.
- **details** (*bool*, optional): if it is set to `True` (default), it gets the models and their details. If `False`, it returns only the models as a *list*.
- **nice_output** (*bool*, optional): if it is set to `True` (default), it prints the models and their details. If `False`, it returns the models as a dictionary (if `details=True`) or a s a list (if `details=False`).

Example: get the list of all the available models and their details:

In [11]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get available models
request.available_models()

CHAOS-6-Core = 'CHAOS-6-Core'(max_degree=20,min_degree=1)
  START: 1997-02-07T05:31:12.000003Z
  END:   2020-04-21T02:49:20.639999Z
DESCRIPTION:
CHAOS Core field
 deprecated model identifier, use CHAOS-Core instead
SOURCES:
  SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701

CHAOS-6-MMA-Primary = 'CHAOS-6-MMA-Primary'(max_degree=2,min_degree=1)
  START: 2012-01-01T00:00:00Z
  END:   2020-02-09T23:30:00Z
DESCRIPTION:
CHAOS Primary (external) magnetospheric field
 deprecated model identifier, use CHAOS-MMA-Primary instead
SOURCES:
  SW_OPER_MMA_CHAOS__20120101T000000_20121231T233000_0608
  SW_OPER_MMA_CHAOS__20130101T000000_20131231T233000_0608
  SW_OPER_MMA_CHAOS__20140101T000000_20141231T233000_0608
  SW_OPER_MMA_CHAOS__20150101T000000_20151231T233000_0608
  SW_OPER_MMA_CHAOS__20160101T000000_20161231T233000_0608
  SW_OPER_MMA_CHAOS__20170101T000000_20171231T233000_0608
  SW_OPER_MMA_CHAOS__20180101T000000_20181231T233000_0701
  SW_OPER_MMA_CHAOS__20190101T000000_20200209T233000

Example: print the list of the available `F` (Fast-Track) models and their details:

In [12]:
request.available_models('F')

MMA_SHA_2F-Primary = 'MMA_SHA_2F-Primary'(max_degree=1,min_degree=1)
  START: 2013-11-25T11:15:00Z
  END:   2020-02-02T21:45:00Z
DESCRIPTION:
[Fast-Track Product]: Primary (external) magnetospheric field
 Rapid modelling of the large-scale magnetospheric field from Swarm satellite data, https://doi.org/10.5047/eps.2013.09.003 
SOURCES:
  SW_OPER_MMA_SHA_2F_20131125T103000_20140101T000000_0108
  SW_OPER_MMA_SHA_2F_20140101T000000_20150101T000000_0108
  SW_OPER_MMA_SHA_2F_20150101T000000_20160101T000000_0108
  SW_OPER_MMA_SHA_2F_20160101T000000_20170101T000000_0108
  SW_OPER_MMA_SHA_2F_20170101T000000_20180101T000000_0108
  SW_OPER_MMA_SHA_2F_20180101T000000_20190101T000000_0108
  SW_OPER_MMA_SHA_2F_20190101T000000_20200101T000000_0108
  SW_OPER_MMA_SHA_2F_20200101T000000_20200202T223000_0108

MMA_SHA_2F-Secondary = 'MMA_SHA_2F-Secondary'(max_degree=1,min_degree=1)
  START: 2013-11-25T11:15:00Z
  END:   2020-02-02T21:45:00Z
DESCRIPTION:
[Fast-Track Product]: Secondary (internal/induced) 

Example: get the If the list of models of type `F` as a dictionary:

In [13]:
request.available_models(param='F', nice_output=False)

{'MMA_SHA_2F-Primary': {'description': ('[Fast-Track Product]: Primary (external) magnetospheric field',
   ' Rapid modelling of the large-scale magnetospheric field from Swarm satellite data, https://doi.org/10.5047/eps.2013.09.003 '),
  'details': {'sources': ['SW_OPER_MMA_SHA_2F_20131125T103000_20140101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20140101T000000_20150101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20150101T000000_20160101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20160101T000000_20170101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20170101T000000_20180101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20180101T000000_20190101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20190101T000000_20200101T000000_0108',
    'SW_OPER_MMA_SHA_2F_20200101T000000_20200202T223000_0108'],
   'expression': "'MMA_SHA_2F-Primary'(max_degree=1,min_degree=1)",
   'validity': {'start': '2013-11-25T11:15:00Z',
    'end': '2020-02-02T21:45:00Z'}}},
 'MMA_SHA_2F-Secondary': {'description': ('[Fast-Track Product]: Secondary (inter

Example: get the list of all the available models without their details:

In [14]:
request.available_models(details=False)

['IGRF12',
 'LCS-1',
 'MF7',
 'CHAOS-Core',
 'CHAOS-Static',
 'CHAOS-MMA-Primary',
 'CHAOS-MMA-Secondary',
 'CHAOS-6-Core',
 'CHAOS-6-Static',
 'CHAOS-6-MMA-Primary',
 'CHAOS-6-MMA-Secondary',
 'MCO_SHA_2C',
 'MCO_SHA_2D',
 'MLI_SHA_2C',
 'MLI_SHA_2D',
 'MMA_SHA_2C-Primary',
 'MMA_SHA_2C-Secondary',
 'MMA_SHA_2F-Primary',
 'MMA_SHA_2F-Secondary',
 'MIO_SHA_2C-Primary',
 'MIO_SHA_2C-Secondary',
 'MIO_SHA_2D-Primary',
 'MIO_SHA_2D-Secondary']

[TOP](#top)

<a id="SwarmRequest.get_model_info"/>

### Get information about one or more models.

It is possible to get information about one or more specific models using the `SwarmRequest.get_model_info()` method:

```python
viresclient.SwarmRequest.get_model_info(models=None, custom_model=None, original_response=False)
```

**Arguments*:

- **models** (*list[str]*, optional): models as a list of strings. If the list is not provided, returns information about all the available magnetic models.
- **custom_model** (*str*, optional): name of the file containing the spherical harmonics coefficients (custom model).
- **original_response** (*bool*, optional): if set to `False` (default), returns the result as a dictionary. If set to `True`, returns the result as a list of dictionaries.

Example: get info about all the available magnetic models:

In [15]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Get model info
request.get_model_info()

{'MLI_SHA_2D': {'sources': ['SW_OPER_MLI_SHA_2D_00000000T000000_99999999T999999_0501'],
  'expression': 'MLI_SHA_2D(max_degree=133,min_degree=16)',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}},
 'MLI_SHA_2C': {'sources': ['SW_OPER_MLI_SHA_2C_00000000T000000_99999999T999999_0501'],
  'expression': 'MLI_SHA_2C(max_degree=120,min_degree=16)',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}},
 'CHAOS-Core': {'sources': ['SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701'],
  'expression': "'CHAOS-Core'(max_degree=20,min_degree=1)",
  'validity': {'start': '1997-02-07T05:31:12.000003Z',
   'end': '2020-04-21T02:49:20.639999Z'}},
 'MIO_SHA_2C-Primary': {'sources': ['SW_OPER_MIO_SHA_2C_00000000T000000_99999999T999999_0501'],
  'expression': "'MIO_SHA_2C-Primary'(max_degree=60,min_degree=1)",
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}},
 'LCS-1': {'sources': ['LCS-1.shc'],
  'ex

Example: get info about CHAOS-Core and CHAOS-Static:

In [16]:
request.get_model_info(models=['CHAOS-Core', 'CHAOS-Static'])

{'CHAOS-Core': {'sources': ['SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701'],
  'expression': "'CHAOS-Core'(max_degree=20,min_degree=1)",
  'validity': {'start': '1997-02-07T05:31:12.000003Z',
   'end': '2020-04-21T02:49:20.639999Z'}},
 'CHAOS-Static': {'sources': ['CHAOS-7_static.shc'],
  'expression': "'CHAOS-Static'(max_degree=185,min_degree=21)",
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}}}

Example: set `original_response=True` to get the result as a list of dictionaries:

In [17]:
request.get_model_info(models=['CHAOS-Core', 'CHAOS-Static'], original_response=True)

[{'sources': ['SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701'],
  'expression': "'CHAOS-Core'(max_degree=20,min_degree=1)",
  'name': 'CHAOS-Core',
  'validity': {'start': '1997-02-07T05:31:12.000003Z',
   'end': '2020-04-21T02:49:20.639999Z'}},
 {'sources': ['CHAOS-7_static.shc'],
  'expression': "'CHAOS-Static'(max_degree=185,min_degree=21)",
  'name': 'CHAOS-Static',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}}]

Example: get info on a custom model by providing the name of the file containing its coefficients:

In [18]:
# Download shc file with WGET (command line tool)
!wget "http://www.spacecenter.dk/files/magnetic-models/LCS-1/LCS-1.shc"

# Upload a .shc file and update the file name
request.get_model_info(custom_model='LCS-1.shc')

--2020-02-11 22:17:14--  http://www.spacecenter.dk/files/magnetic-models/LCS-1/LCS-1.shc
Resolving www.spacecenter.dk (www.spacecenter.dk)... 130.226.216.199
Connecting to www.spacecenter.dk (www.spacecenter.dk)|130.226.216.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 657666 (642K) [text/plain]
Saving to: ‘LCS-1.shc’

LCS-1.shc           100%[===================>] 642.25K  3.57MB/s    in 0.2s    

2020-02-11 22:17:15 (3.57 MB/s) - ‘LCS-1.shc’ saved [657666/657666]



{'Custom_Model': {'sources': [],
  'expression': 'Custom_Model(max_degree=185,min_degree=1)',
  'validity': {'start': '0001-01-01T00:00:00Z',
   'end': '4000-01-01T00:00:00Z'}}}

In [19]:
# Delete downloaded file(s)
!rm LCS-1.shc*

[TOP](#top)

<a id="SwarmRequest.get_orbit_number"/>

### Get the orbit number

The `SwarmRequest.get_orbit_number()` allows to get the orbit number of a given spacecraft providing date and time:

```python
viresclient.SwarmRequest.get_orbit_number(spacecraft, input_time)
```

**Arguments**:

- **spacecraft** (*str*): spacecraft identifier: `A`, `B` or `C`.
- **input_time** (*datetime.datetime* or *str*): date and time.

Example: get orbit numbers corresponding to date 2020-01-01 00:00:00 for the three spacecrafts:

In [24]:
# Create the request object
request = SwarmRequest()

# Get results
for sc in ('A', 'B', 'C'):
    orbit = request.get_orbit_number(sc, '2020-01-01T00:00:00')
    print(f's/c {sc}: {orbit}')

s/c A: 34324
s/c B: 33892
s/c C: 34320


[TOP](#top)

<a id="SwarmRequest.get_times_for_orbits"/>

### Get times for orbits

Get the time interval corresponding to a pair of orbit numbers using the `SwarmRequest.get_times_for_orbits()` method:

```python
viresclient.SwarmRequest.get_times_for_orbits(spacecraft, start_orbit, end_orbit)
```

**Arguments**:

- **spacecraft** (*str*): spacecraft identifier: `A`, `B` or `C`.
- **start_orbit** (*int*): start orbit number
- **end_orbit** (*int*): end orbit number

Example: get time intervals corresponding to *start_orbit* 1000 and *end_orbit* 2000 for the three spacecrafts:

In [25]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Set start/end orbits
start_orbit = 1000
end_orbit = 2000

# Create the request object
request = SwarmRequest()

# Get results
for sc in ('A', 'B', 'C'):
    start_date, end_date = request.get_times_for_orbits(sc, start_orbit, end_orbit)
    print(f's/c {sc}: {start_date} - {end_date}')

s/c A: 2014-01-27 04:05:04.228591 - 2014-04-02 13:51:01.876037
s/c B: 2014-01-27 04:04:17.158156 - 2014-04-03 00:46:19.552254
s/c C: 2014-01-27 04:08:31.126684 - 2014-04-02 19:45:08.511104


[TOP](#top)

<a id="SwarmRequest.set_collection"/>

### Set collections

Before sending the request to the server, you need to set the Swarm collection including the measurement(s) of interest. This can be done using the `SwarmRequest.set_collection()` method:

```python
viresclient.SwarmRequest.set_collection(*args)
```

**Arguments**:

- ***args** (*str*): one or more collections (see `SwarmRequest.available_collections()`) as a string.

Example: to get data from `SW_OPER_MAGA_LR_1B` and `SW_OPER_EFIA_LP_1B` collections:

In [26]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

[TOP](#top)

<a id="SwarmRequest.set_products"/>

### Set products

After setting the collection, you must set the combination of measurements and/or auxiliaries and/or magnetic model(s) data to retrieve. This can be done using the `SwarmRequest.set_products()` method:

```python
viresclient.SwarmRequest.set_products(measurements=None, models=None, custom_model=None, auxiliaries=None, residuals=False, sampling_step=None)
```

**Arguments**:

- **measurements** (*list[str]*, optional): list of measurements to be downloaded. To get the list of the available measurements see [Available measurements](#SwarmRequest.available_measurements) (e.g.: `['F', 'B_NEC', 'Ne']`.).
- **models** (*list[str]*, optional): list of magnetic models. To get the list of the available models see [Available models](#SwarmRequest.available_models) (e.g.: `['CHAOS-Core', 'CHAOS-Static']`). In addition to the list, this parameters accepts also expression for the definition of magnetic models (e.g.: `'CHAOS = "CHAOS-Core" + "CHAOS-Static"'`).
- **custom_model** (*str*, optional): path to the the file containing the spherical harmonics coefficients of the custom model.
- **auxiliaries** (*list[str]*, optional): list of auxiliaries to be downloaded. To get the list of the available auxiliaries see [Available auxiliaries](#SwarmRequest.available_auxiliaries). Please note that the following parameters are always retrieved (i.e. they don't need to be specified): `Spacecraft`, `Timestamp`, `Latitude`, `Longitude`, `Radius`.
- **residuals** (*bool*, optional): if it is set to `True`, returns the residuals between measurements (specified with *measurements*) and models (specified with *models*). If it is set to `False` (default), returns measurements and models.
- **sampling_step** (*str*, optional): set the sampling step as an [ISO 8601 time interval](https://en.wikipedia.org/wiki/ISO_8601#Time_intervals). If not provided, data is returned with the original sampling.


Example: get measurements: `F` and `B_NEC` from `SW_OPER_MAGA_LR_1B` and `Ne` from `SW_OPER_EFIA_LP_1B`, model `CHAOS = "CHAOS-Core" + "Chaos-Static"` and auxiliary `OrbitNumber` with a sampling step of 10 seconds:

In [27]:
# Import SwarmRequest object (this step can be skipped if SwarmRequest has been already imported)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

# Set products
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)

[TOP](#top)

<a id="SwarmRequest.set_range_filter"/>

### Set/clear filters

Filter(s) can be applied to the requested measurements using the `SwarmRequest.set_range_filter()` method:

```python
viresclient.SwarmRequest.set_range_filter(parameter=None, minimum=None, maximum=None)
```

This method allows to set filter(s) in the form: $minimum \le parameter \le maximum$.

**Arguments**:

- **parameter** (*str*, optional): parameter to be used as a filter (e.g. `Latitude`)
- **minimum** (*float*, optional): allowed minimum value
- **maximum** (*float*, optional): allowed maximum value

It is possible to apply multiple filters with consecutive calls to this method.

<a id="example1"/>

**Example 1:** to set filter: -20 <= `Longitude` <= 50 and 30 <= `Latitude` <= 70:

In [28]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collection
request.set_collection('SW_OPER_MAGA_LR_1B')

# Set product
request.set_products(measurements=['F', 'B_NEC'])

# Set filters
request.set_range_filter('Longitude', -20.0, 50.0)
request.set_range_filter('Latitude', 30.0, 70.0)

Filters can be removed using the `SwarmRequest.clear_range_filter()` method:

```python
viresclient.SwarmRequest.clear_range_filter()
```

In [29]:
request.clear_range_filter()

[TOP](#top)

<a id="SwarmRequest.get_between"/>

### Send request to the server

After setting collection(s), measurements, auxiliaries and models, we are ready to send the request to the server using the `SwarmRequest.get_between()` method:

```python
viresclient.SwarmRequest.get_between(start_time=None, end_time=None, filetype='cdf', asynchronous=True, show_progress=True, nrecords_limit=None, tmpdir=None)
```

**Arguments**:

- **start_time** (*datetime.datetime* or *str*, optional): lower bound of temporal interval. If provided as string, it must be compliant to [ISO-8601](https://en.wikipedia.org/wiki/ISO_8601).
- **end_time** (*datetime.datetime* or *str*, optional): upper bound of temporal interval. If provided as string, it must be compliant to [ISO-8601](https://en.wikipedia.org/wiki/ISO_8601).
- **filetype** (*str*, optional): file format. Allowed values: `csv` and `cdf` (default).
- **asynchronous** (*bool*, optional): if `True` (default), set the asynchronous processing.
- **show_progress** (*bool*, optional): if `True` (default), enable the progress bar while processing and downloading data.
- **nrecords_limit** (*int*): overrides the limit of 3456000 records.


<a id="example2"/>

**Example 2:** get F and B_NEC from SW_OPER_MAGA_LR_1B, Ne from SW_OPER_EFIA_LP_1B, model CHAOS = "CHAOS-Core" + "Chaos-Static" and auxiliary OrbitNumber with a sampling step of 10 seconds, between 2019-10-01T00:00:00 and 2019-10-01T01:00:00 in CDF format:

In [30]:
# Import SwarmClient (this step can be omitted if already executed in the previous examples)
from viresclient import SwarmRequest

# Create the request object
request = SwarmRequest()

# Set collections
request.set_collection('SW_OPER_MAGA_LR_1B', 'SW_OPER_EFIA_LP_1B')

# Set products
request.set_products(
    measurements=['F', 'B_NEC', 'Ne'],
    models=['CHAOS = "CHAOS-Core" + "CHAOS-Static"'],
    auxiliaries=['OrbitNumber'],
    residuals=False,
    sampling_step='PT10S'
)

# Get data
data = request.get_between('2019-10-01T00:00:00', '2019-10-01T01:00:00')

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.098MB)


Data is returned as a `ReturnedData` object:

In [31]:
type(data)

viresclient._data_handling.ReturnedData

[TOP](#top)

<a id="ReturnedData"/>

## Handle downloaded data - ReturnedData

Once the server receives the user's request, it creates the product, than this product is automatically downloaded and returned to the client as a `ReturnedData` object. Thus you don't need to create it by yourself. It is now possible to convert this data to a `pandas.DataFrame` object or to a `xarray.Dataset` object or to save it to one or more files.

`ReturnedData` object has the following attributes:

- `ReturnedData.sources`
- `ReturnedData.contents`
- `ReturnedData.file_types`
- `ReturnedData.magnetic_models`
- `ReturnedData.range_filters`

and the following methods:

- `ReturnedData.as_dataframe()`
- `ReturnedData.as_xarray()`
- `ReturnedData.to_file()`
- `ReturnedData.to_files()`


[TOP](#top)

<a id="ReturnedData.sources"/>

### Get the list of source data

This attribute contains the list of source data files from which the values have been extracted.

After executing code at [Example 2](#example2), let's see the data sources:

In [7]:
data.sources

['CHAOS-7_static.shc',
 'SW_OPER_AUXAORBCNT_20131122T000000_20200207T000000_0001',
 'SW_OPER_EFIA_LP_1B_20190930T000000_20190930T235959_0501_MDR_EFI_LP',
 'SW_OPER_EFIA_LP_1B_20191001T000000_20191001T235959_0501_MDR_EFI_LP',
 'SW_OPER_MAGA_LR_1B_20191001T000000_20191001T235959_0505_MDR_MAG_LR',
 'SW_OPER_MCO_SHA_2X_19970101T000000_20200419T235959_0701']

This is the list of the files from which the measurement and auxiliaries values and the magnetic models values have been retrieved.

[TOP](#top)

<a id="ReturnedData.contents"/>

### ReturnedData contents

Downloaded data is saved to one or more temporary files represente as `ReturnedDataFile` objects. The `ReturnedData.contents` attribute contains the list of these objects.

After executing code at [Example 2](#example2), let's see the data contents:

In [8]:
data.contents

[TOP](#top)

<a id="ReturnedData.filetype"/>

### Get type of downloaded data files

This attribute contains the type of downloaded files (i.e. `cdf` or `csv`).

After executing code at [Example 2](#example2), let's the file type:

In [9]:
data.filetype

'cdf'

[TOP](#top)

<a id="ReturnedData.magnetic_models"/>

### Get list of magnetic models used during calculations

This attribute contains the list of the magnetic models used during calculations.

After executing code at [Example 2](#example2), let's see the list of model(s):

In [10]:
data.magnetic_models

["CHAOS = 'CHAOS-Core'(max_degree=20,min_degree=1) + 'CHAOS-Static'(max_degree=185,min_degree=21)"]

[TOP](#top)

<a id="ReturnedData.range_filters"/>

### Get list of filters applied to the request

This attribute contains the list of applied filters

After executing code at [Example 1](#example1), let's see the applied filters:

In [20]:
# Get data between '2020-01-01T00:00:00' and '2020-01-02T00:00:00'
data = request.get_between('2020-01-01T00:00:00', '2020-01-02T00:00:00')

# Get list of applied filters
data.range_filters

[1/1] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (0.249MB)


['Latitude:30,70', 'Longitude:-20,50']

After executing code at [Example 2](#example2), let's see the applied filters:

In [22]:
data.range_filters

[]

[TOP](#top)

<a id="ReturnedData.as_dataframe"/>

### Convert ReturnedData to Pandas DataFrame

Data downloaded from the server can be converted to a `pandas.DataFrame` object. This is a general 2D labeled, size-mutable tabular structure with potentially heterogeneously-typed column allowing the user to directly access data. For more information:

- https://pandas.pydata.org/pandas-docs/stable/
- https://pandas.pydata.org/pandas-docs/stable/reference/frame.html

This conversion can be obtained with the `ReturnedData.as_dataframe()` method:

```python
ReturnedData.as_dataframe(expand=False)
```

The optional *expand* parameter is, by default, set to `False`. This means that the vector parameters are represented as arrays (i.e. all the vector components in the same column). If this parameter is stet to `True`, the vector parameters are expanded (i.e. each component in a separate column). 

After executing code at [Example 2](#example2), let's convert the `ReturnedData` to a `DataFrame`:

In [24]:
data.as_dataframe().head()

,Spacecraft,Latitude,Longitude,Radius,F,F_CHAOS,B_NEC,B_NEC_CHAOS,Ne,OrbitNumber
2019-10-01 00:00:00,A,-33.130184,37.641386,6820061.65,26246.0328,26240.985891,"[11030.816, -6222.3955000000005, -22988.693300...","[11046.424869499242, -6221.021024854402, -2297...",108830.9,32904
2019-10-01 00:00:10,A,-33.771114,37.642219,6820113.72,26281.9878,26276.579667,"[10910.8961, -6345.64, -23053.250200000002]","[10925.955568855303, -6344.535396315394, -2303...",116231.7,32904
2019-10-01 00:00:20,A,-34.412011,37.643695,6820164.78,26322.3415,26316.580822,"[10798.207900000001, -6468.446800000001, -2311...","[10812.911047129362, -6467.100084355649, -2310...",116226.5,32904
2019-10-01 00:00:30,A,-35.052873,37.645838,6820214.83,26367.2575,26361.164222,"[10692.824700000001, -6590.2154, -23183.793700...","[10707.112068975179, -6588.730009330973, -2317...",120598.5,32904
2019-10-01 00:00:40,A,-35.693700,37.648675,6820263.88,26416.9227,26410.501735,"[10594.3256, -6710.861, -23250.8852]","[10608.360249796353, -6709.418192882664, -2323...",129792.6,32904


Setting `expand=True`:

In [25]:
data.as_dataframe(expand=True).head()

,Spacecraft,Latitude,Longitude,Radius,F,F_CHAOS,B_NEC_N,B_NEC_E,B_NEC_C,B_NEC_CHAOS_N,B_NEC_CHAOS_E,B_NEC_CHAOS_C,Ne,OrbitNumber
2019-10-01 00:00:00,A,-33.130184,37.641386,6820061.65,26246.0328,26240.985891,11030.8160,-6222.3955,-22988.6933,11046.424869,-6221.021025,-22975.307083,108830.9,32904
2019-10-01 00:00:10,A,-33.771114,37.642219,6820113.72,26281.9878,26276.579667,10910.8961,-6345.6400,-23053.2502,10925.955569,-6344.535396,-23039.726659,116231.7,32904
2019-10-01 00:00:20,A,-34.412011,37.643695,6820164.78,26322.3415,26316.580822,10798.2079,-6468.4468,-23118.1299,10812.911047,-6467.100084,-23104.544950,116226.5,32904
2019-10-01 00:00:30,A,-35.052873,37.645838,6820214.83,26367.2575,26361.164222,10692.8247,-6590.2154,-23183.7937,10707.112069,-6588.730009,-23170.182717,120598.5,32904
2019-10-01 00:00:40,A,-35.693700,37.648675,6820263.88,26416.9227,26410.501735,10594.3256,-6710.8610,-23250.8852,10608.360250,-6709.418193,-23237.060964,129792.6,32904


[TOP](#top)

<a id="ReturnedData.as_xarray"/>

### Convert ReturnedData to xarray Dataset

Data downloaded from the server can be converted to a `xarray.Dataset` object. This is a multi-dimentional array allowing the user to directly access data. For more information:

- http://xarray.pydata.org/en/stable/
- http://xarray.pydata.org/en/stable/data-structures.html#dataset

This conversion can be obtained with the `ReturnedData.as_xarray()`.

After executing code at [Example 2](#example2), let's convert the `ReturnedData` to a `Dataset`:

In [26]:
data.as_xarray()

<xarray.Dataset>
Dimensions:      (NEC: 3, Timestamp: 360)
Coordinates:
  * Timestamp    (Timestamp) datetime64[ns] 2019-10-01 ... 2019-10-01T00:59:50
Dimensions without coordinates: NEC
Data variables:
    Spacecraft   (Timestamp) <U1 'A' 'A' 'A' 'A' 'A' 'A' ... 'A' 'A' 'A' 'A' 'A'
    Latitude     (Timestamp) float64 -33.13 -33.77 -34.41 ... 82.09 82.69 83.29
    Longitude    (Timestamp) float64 37.64 37.64 37.64 ... -139.6 -137.9 -135.9
    Radius       (Timestamp) float64 6.82e+06 6.82e+06 ... 6.803e+06 6.803e+06
    F            (Timestamp) float64 2.625e+04 2.628e+04 ... 4.749e+04 4.746e+04
    F_CHAOS      (Timestamp) float64 2.624e+04 2.628e+04 ... 4.748e+04 4.745e+04
    B_NEC        (Timestamp, NEC) float64 1.103e+04 -6.222e+03 ... 4.744e+04
    B_NEC_CHAOS  (Timestamp, NEC) float64 1.105e+04 -6.221e+03 ... 4.744e+04
    Ne           (Timestamp) float64 1.088e+05 1.162e+05 ... 4.102e+04 3.138e+04
    OrbitNumber  (Timestamp) int32 32904 32904 32904 32904 ... 32905 32905 32905
Attributes:
    Sources:         ['CHAOS-7_static.shc', 'SW_OPER_AUXAORBCNT_20131122T0000...
    MagneticModels:  ["CHAOS = 'CHAOS-Core'(max_degree=20,min_degree=1) + 'CH...
    RangeFilters:    []

[TOP](#top)

<a id="ReturnedData.to_file"/>

### Save downloaded data to a file

Data downloaded from the server can be saved to a file using the `ReturnedData.to_file()` method:

```python
ReturnedData.to_file(path, overwrite=False)
```


Arguments:

- **path** (*str*): output file path.
- **overwrite** (*bool*, optional):if `True` allows to overwrite the file if already present at *path*.


After executing code at [Example 2](#example2), let's save the `ReturnedData` to file:

In [28]:
data.to_file('out.cdf', overwrite=True)

Data written to out.cdf


**Note**: this method can be used only if the amount of downloaded data is small (i.e. if the request is not split between multiple requests).

<a id="example3"/>

**Example 3**: try to download `U_orbit`, `Ne`, `Te`, `Vs` EFI data between 2019-10-01T00:00:00' and '2019-11-01T00:00:00':

In [35]:
# Set SW_OPER_EFIA_LP_1B collection (for EFIA_LP_1B product type)
request.set_collection('SW_OPER_EFIA_LP_1B')

# Set measurements
request.set_products(measurements=['U_orbit', 'Ne', 'Te', 'Vs'])

# Get data
data = request.get_between('2019-10-01T00:00:00', '2019-11-01T00:00:00')

[1/2] Processing:  100%|██████████|  [ Elapsed: 00:03, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:02, Remaining: 00:00 ] (279.743MB)
[2/2] Processing:  100%|██████████|  [ Elapsed: 00:01, Remaining: 00:00 ]
      Downloading: 100%|██████████|  [ Elapsed: 00:00, Remaining: 00:00 ] (67.389MB)


The request is split between multiple requests. Try to execute the cell below:

In [ ]:
data.to_file('huge.cdf', True)

In [32]:
# Remove saved files (if any)
!rm *.cdf

[TOP](#top)

<a id="ReturnedData.to_files"/>

### Save downloaded data to multiple files

Data downloaded from the server can be saved to one or more files using the `ReturnedData.to_files()` method:

```python
ReturnedData.to_files(paths, overwrite=False)
```

Arguments:

- **paths** (*list[str]*): output files path as a list of strings.
- **overwrite** (*bool*, optional):if `True` allows to overwrite the file if already present at *path*.


With reference to [Example 2](#example2), let's save the data to a single file:

In [34]:
data.to_files(['out.cdf'], overwrite=True)

Data written to out.cdf


This method is very useful in case the request has been split between multiple requests. With reference to [Example 3](#example3) (execute again the code), let's save the downloaded data in two files:

In [36]:
data.to_files(['first.cdf', 'second.cdf'], overwrite=True)

Data written to first.cdf
Data written to second.cdf


**Note**: the number of files to be specified must be equal to the number of files indicated by the `ReturnedData.contents` attribute. In the above case:

In [37]:
data.contents

In [39]:
# Remove saved files (if any)
!rm *.cdf

[TOP](#top)

<a id="ReturnedDataFile"/>

## Handle downloaded temporary data file - ReturnedDataFile

This object holds the file downloaded from the server. Even if data has not ben saved to a file with the `ReturnedData.to_files()`, it is stored in a temporary file and automatically deleted when not needed anymore. As indicated in the "[ReturnedData contents](#ReturnedData.contents)" section, you can get the list of the returned data files using the `ReturnedData.contents` attribute. Thus, you don't need to create this object by yourself.

**Note**: the description of this object has been included for completeness only. You won't need to use this object and its methods directly. To handle the downloaded data is preferable to use the [ReturnedData](#ReturnedData) object. 

`ReturnedDataFile` object has the following attributes:

- `ReturnedDataFile._file`
- `ReturnedDataFile.filetype`

and the following methods:

- `ReturnedDataFile.as_dataframe()`
- `ReturnedDataFile.as_xarray()`
- `ReturnedDataFile.to_file()`

[TOP](#top)

<a id="ReturnedDataFile._file"/>

### Get the NamedTemporaryFile associated to a ReturnedDataFile

The `NamedTemporaryFile` corresponding to the `ReturnedDataFile` is contained in the `ReturnedDataFile._file` attribute.

After executing code at [Example 2](#example2), let's get the list of `ReturnedDataFile` objects:

In [13]:
data.contents

`ReturnedData` contains only one `ReturnedDataFile`. Let's get the associated `NamedTemporaryFile`:

In [14]:
data.contents[0]._file

`tempfile.NamedTemporaryFile` is part of the Python standard library. For more information see: https://docs.python.org/3/library/tempfile.html#tempfile.NamedTemporaryFile

[TOP](#top)

<a id="ReturnedDataFile.filetype"/>

### Get type of the downloaded data file

This attribute contains the type of downloaded files (i.e. `cdf`, `csv` or `nc`)

Aftere executing code at [Example 2](#example2):

In [15]:
data.contents[0].filetype

'cdf'

[TOP](#top)

<a id="ReturnedDataFile.as_dataframe"/>

### Convert ReturnedDataFile to Pandas Dataframe

As for the `ReturnedData` object (see [Convert ReturnedData to Pandas DataFrame](#ReturnedData.as_dataframe)), the `ReturnedDataFile` object can be converted to a Pandas DataFrame using the `ReturnedDataFile.as_dataframe()` method:

```python
ReturnedDataFile.as_dataframe(expand=False)
```

The optional `expand` parameter is, by default, set to `False`. This means that the vector parameters are presented as arrays (i.e. all the vector components in a single column). If it is set to `True`, the vector parameters are expanded with each componet in a different column.

[TOP](#top)

<a id="ReturnedDataFile.as_xarray"/>

### Convert ReturnedDataFile to xarray Dataset

As for the `ReturnedData` object (see [Convert ReturnedData to xarray Dataset](#ReturnedData.as_xarray)), the `ReturnedDataFile` object can be converted to an xarray Dataset using the `ReturnedDataFile.as_xarray()` method:

```python
ReturnedDataFile.as_xarray()
```

[TOP](#top)

<a id="ReturnedDataFile.to_file"/>

### Save ReturnedDataFile object to a file

Data stored in the `ReturnedDataFile` object can be saved to a file with the `ReturnedDataFile.to_file()` method:

```python
ReturnedDataFile.to_file(path, overwrite=False)
```

Arguments:

- **path** (*str*): output file path.
- **overwrite** (*bool*, optional):if `True` allows to overwrite the file if already present at *path*.


Aftere executing code at [Example 2](#example2):

In [17]:
data.contents[0].to_file('out.cdf', overwrite=True)

Data written to out.cdf


In [18]:
# Remove saved files (if any)
!rm out.cdf

[TOP](#top)

<a id="ClientConfig"/>

## Handle viresclient configuration - ClientConfig

You can acces the `viresclient` configuration using the `ClientConfig` class:

```python
class viresclient.ClientConfig(path=None)
```

**Arguments**:
- **path** (*str*, optional): path of the configuration file. If not specified, the default configuration file is assumed: `~/.viresclient.ini`.

<a id="example4"/>

**Example 4**: create the `ClientConfig` object associated to the default configuration file:

In [1]:
# Import the class
from viresclient import ClientConfig

# Create the ClientConfig object
default = ClientConfig()

`ClientConfig` object has the following attributes:

- `ClientConfig.path`
- `ClientConfig.default_url`

and the following methods:

- `ClientConfig.set_site_config()`
- `ClientConfig.get_site_config()`
- `ClientConfig.save()`

[TOP](#top)

<a id="ClientConfig.path"/>

### Get path of the configuration file

The `ClientConfig.path` read-only attribute contains the path of the configuration file.

With reference to [Example 4](#example4):

In [3]:
default.path

'/home/jovyan/.viresclient.ini'

[TOP](#top)

<a id="ClientConfig.default_url"/>

### Get or set the default URL

The `ClientConfig.default_url` attribute contains server's default URL (i.e. the one used when `SwarmClient` class is invoked without URL).

With reference to [Example 4](#example4):

In [5]:
default.default_url

'https://vires.services/ows'

If the default URL is not set, the attribute returns `None`.

<a id="example5"/>

**Example 5**: create a new configuration:

In [3]:
# Create new configuration
newcfg = ClientConfig('newfile.ini')

# Print default URL 
print(newcfg.default_url)

None


The attribute can be used also to set the default URL, e.g.:

In [4]:
newcfg.default_url = 'https://vires.services/ows'

[TOP](#top)

<a id="ClientConfig.set_site_config"/>

### Set site configuration

It is possible to set the configuration for a server identified by an URL with the `set_site_config()` method:

```python
viresclient.ClientConfig.set_site_config(url, **options)
```

**Arguments**:

- **url** (*str*): server URL
- ****options** (*str*): configuration options in the form: *key*=*value* (e.g.: token='...')

In [Example 5](#example5) we have created a new configuration with a default URL. Let's add a token for the server identified by the default URL.

In [5]:
newcfg.set_site_config(newcfg.default_url, token='mytokenstring')

[TOP](#top)

<a id="ClientConfig.get_site_config"/>

### Get site configuration

It is possible to get the configuration for a server identified by an URL with the `get_site_config()` method:

```python
viresclient.ClientConfig.get_site_config(url)
```

**Arguments**:

- **url** (*str*): server URL

With reference to [Example 5](#example5), get the configuration for `https://vires.services/ows`:

In [7]:
# Set configuration (this step can be avoided if it has already been executed)
newcfg.set_site_config(newcfg.default_url, token='mytokenstring')

# Get the configuration
newcfg.get_site_config('https://vires.services/ows')

{'token': 'mytokenstring'}

[TOP](#top)

<a id="ClientConfig.save"/>

### Save configuration

The configuration stored in the `ClientConfig` object can be saved using the `save()` method:

```python
viresclient.ClientConfig.save()
```

This method saves the configuration to the path specified during the `ClientConfig` creation. You can check this value via the `ClientConfig.path` attribute.

With reference to [Example 5](#example5), get the configuration for `https://vires.services/ows`:

In [8]:
# Set configuration (this step can be avoided if it has already been executed)
newcfg.set_site_config(newcfg.default_url, token='mytokenstring')

# Save the configuration
newcfg.save()

Let's have a look to the new configuration file:

In [9]:
!cat newfile.ini

[default]
url = https://vires.services/ows

[https://vires.services/ows]
token = mytokenstring



In [10]:
# delete newfile.ini
!rm newfile.ini

[TOP](#top)

<a id="DataUpload"/>

## Upload data to the server - DataUpload

You can upload your data to the server to view it in the VirES web interface. File format can be CDF or CSV. Variable names and types must be compliant to: https://github.com/ESA-VirES/VirES-Server/blob/master/vires/custom_data_format_description.md.

Data can be uploaded using the `DataUpload` object:

```python
class viresclient.DataUpload(url, token, **kwargs)
```

**Arguments**:

- **url** (*str*): server URL
- **token** (*str*): access token
- ****kwargs** (*str*): additional parameters (currently not used)

`DataUpload` object has the following attributes:

- `DataUpload.ids`

and the following methods:

- `DataUpload.post()`
- `DataUpload.get()`
- `DataUpload.set_constant_parameters()`
- `DataUpload.get_constant_parameters()`

<a id="example6"/>

**Example 6**: create a `DataUpload` object for data upload to the default server. We can retrieve the default URL and the access token from the configuration, using the `ClientConfig` object:

In [12]:
# Import DataUpload object
from viresclient import DataUpload

# Import ClientConfig object (this step can be avoided if ClientConfig has been already imported)
from viresclient import ClientConfig

# Create ClientConfig object associated to the default configuration file: ~/.viresclient.ini
default = ClientConfig()

# Get default URL and access token from the configuration
url = default.default_url
token = default.get_site_config(url)['token']

# Create the DataUpload object:
du = DataUpload(url, token)

[TOP](#top)

<a id="DataUpload.post"/>

### Upload a file to the server

You can upload a file to the server using the `DataUpload.post()` method:

```python
viresclient.DataUpload.post(file, filename=None)
```

**Arguments**:

- **file** (*str*): file to be uploaded

The method returns the info about the uploaded file as a dictionary.

After executing code in [Example 2](#example2) and in [Example 6](#example6):

In [9]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Data written to out.cdf


Let's see the info about the uploaded file:

In [10]:
info

{'missing_fields': {},
 'source_fields': ['OrbitNumber',
  'F',
  'Timestamp',
  'B_NEC_CHAOS',
  'F_CHAOS',
  'Ne',
  'Longitude',
  'B_NEC',
  'Radius',
  'Latitude'],
 'content_type': 'application/x-cdf',
 'owner': 'demo',
 'size': 98031,
 'end': '2019-10-01T00:59:50Z',
 'constant_fields': {},
 'data_file': 'out.cdf',
 'fields': {'F': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Timestamp': {'shape': [], 'cdf_type': 31, 'data_type': 'CDF_EPOCH'},
  'Latitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC_CHAOS': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'F_CHAOS': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Ne': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Longitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Radius': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'OrbitNumber': {'shape': [], '

In [6]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.ids"/>

### Get the identifier(s) of the uploaded file(s)

You can obtain the identifiers of the uploaded files via the `DataUpload.ids` attribute as a list. Please note that currently the server accepts only one file at a time.

After executing code in [Example 2](#example2) and in [Example 6](#example6):

In [6]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Data written to out.cdf


Get id(s) of the uploaded file(s):

In [5]:
du.ids

['34a1a590-0ad4-4e8a-82cd-eaf4439be9b5']

In [6]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.get"/>

### Get info about the uploaded file

You can get the info of the uploaded file using the `DataUpload.get()` method:

```python
viresclient.DataUpload.get(identifier=None)
```

**Attributes**

- **identifier** (*str*, optional): identifier of the uploaded file. If not provided, returns the info af all the uploaded files as a list.

After executing code in [Example 2](#example2) and in [Example 6](#example6):

In [7]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Data written to out.cdf


Get info about the uploaded file:

In [9]:
du.get(info['identifier'])

{'missing_fields': {},
 'source_fields': ['OrbitNumber',
  'F',
  'Timestamp',
  'B_NEC_CHAOS',
  'F_CHAOS',
  'Ne',
  'Longitude',
  'B_NEC',
  'Radius',
  'Latitude'],
 'content_type': 'application/x-cdf',
 'owner': 'demo',
 'size': 98031,
 'end': '2019-10-01T00:59:50Z',
 'constant_fields': {},
 'data_file': 'out.cdf',
 'fields': {'F': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Timestamp': {'shape': [], 'cdf_type': 31, 'data_type': 'CDF_EPOCH'},
  'Latitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC_CHAOS': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'F_CHAOS': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Ne': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Longitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'B_NEC': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'Radius': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
  'OrbitNumber': {'shape': [], '

If the identifier is not provided, get info about all the files as a list:

In [10]:
du.get()

[{'missing_fields': {},
  'source_fields': ['OrbitNumber',
   'F',
   'Timestamp',
   'B_NEC_CHAOS',
   'F_CHAOS',
   'Ne',
   'Longitude',
   'B_NEC',
   'Radius',
   'Latitude'],
  'content_type': 'application/x-cdf',
  'owner': 'demo',
  'size': 98031,
  'end': '2019-10-01T00:59:50Z',
  'constant_fields': {},
  'data_file': 'out.cdf',
  'fields': {'F': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Timestamp': {'shape': [], 'cdf_type': 31, 'data_type': 'CDF_EPOCH'},
   'Latitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'B_NEC_CHAOS': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'F_CHAOS': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Ne': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Longitude': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'B_NEC': {'shape': [3], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Radius': {'shape': [], 'cdf_type': 45, 'data_type': 'CDF_DOUBLE'},
   'Or

In [6]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.set_constant_parameters"/>

### Set constant parameters to the uploaded file

It is possible to set constant parameters to the uploaded file using the `DataUpload.set_constant_parameters()` method:

```python
viresclient.DataUpload.set_constant_parameters(identifier, parameters, replace=False)
```

**Arguments**:

- **identifier** (*str*): file identifier.
- **parameters** (*dict*): constant parameters provided as a dictionary
- **replace** (*bool*, optional): if set to `True`, all the parameters will be replaced by the new parameters, otherwise the new parameters will update the existing ones (default behaviour).


After executing code in [Example 2](#example2) and in [Example 6](#example6):

In [13]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Data written to out.cdf


Assign constant parameters: `param1 = 12345` and `param2 = 34567`:

In [14]:
du.set_constant_parameters(du.ids[0], {'param1': 12345, 'param2': 34567})

{'param2': 34567, 'param1': 12345}

If we want to set `param1` to a new value we can update the existing parameters:

In [15]:
du.set_constant_parameters(du.ids[0], {'param1': 1})

{'param2': 34567, 'param1': 1}

or replace all the parameters:

In [16]:
du.set_constant_parameters(du.ids[0], {'param1': 1}, replace=True)

{'param1': 1}

Note that `param2` does not exist anymore.

In [6]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.get_constant_parameters"/>

### Get constant parameters applied to the uploaded file

It is possible to get the list of constant parameters applied to the uploaded file using the `DataUpload.get_constant_parameters()` method:

```python
viresclient.DataUpload.get_constant_parameters(identifier)
```

**Arguments**:

- **identifier** (*str*): file identifier.


After executing code in [Example 2](#example2) and in [Example 6](#example6):

In [17]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Data written to out.cdf


Assign constant parameters: `param1 = 12345` and `param2 = 34567`:

In [18]:
du.set_constant_parameters(du.ids[0], {'param1': 12345, 'param2': 34567})

{'param2': 34567, 'param1': 12345}

Get the list of constant parameters:

In [19]:
du.get_constant_parameters(du.ids[0])

{'param2': 34567, 'param1': 12345}

In [20]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.delete"/>

### Delete a specific uploaded file

You can delete a specific uploaded file using the `DataUpload.delete()` method:

```python
viresclient.DataUpload.delete(identifier)
```

**Arguments**:

- **identifier** (*str*): file identifier.


E.g. to delete the file previously uploaded:

In [115]:
du.delete(du.ids[0])

du.ids

[]

In [6]:
# Delete test file (if any)
!rm out.cdf

[TOP](#top)

<a id="DataUpload.clear"/>

### Delete the uploaded files

You can delete all the uploaded files using the `DataUpload.clear()` method:

```python
viresclient.DataUpload.clear()
```

After executing code in [Example 2](#example2) and in [Example 6](#example6):

In [21]:
# Save ReturnedData to a file
data.to_file('out.cdf', overwrite=True)

# Upload the file to the server to be visualized in the web client
info = du.post('out.cdf')

Data written to out.cdf


In [22]:
du.clear()

du.ids

[]